In [26]:
import torch
from torch import nn
import torch.nn.functional as F

torch.manual_seed(87)
torch.cuda.manual_seed(87)

Setup

In [27]:
import numpy as np
from sklearn.datasets import make_classification

np.random.seed(87)

X, y = make_classification(
    n_samples=1000, n_features=20, n_informative=10, random_state=87)
X, y = X.astype(np.float32), y.astype(np.int64)

print(X.shape, y.shape, y.mean())

(1000, 20) (1000,) 0.496


Define pytorch classification module

In [28]:
from skorch import NeuralNetClassifier

class ClassifierModule(nn.Module):
    def __init__(
        self,
        num_units=10,
        nonlin=F.relu,
        dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(20, num_units)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)
        self.dropout = nn.Dropout(dropout)
        self.nonlin = nonlin
        
    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        
        X = self.nonlin(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X

Writing CUSTOM callback
Rules:
    1. inherit skorch.callbacks.Callback
    2. implement atleast one on_ method
    3. argments first neuralnet instance, second optionally local data, **kwargs
    (4. attributes that should be reset in the initialize method

mock twitter API that tweets epoch validation accuracy

In [29]:
from skorch.callbacks import Callback

def tweet(msg):
    print('~'*60)
    print("*tweet*", msg, "#skorch #pytorch")
    
class AccuracyTweet(Callback):
    def __init__(self, min_accuracy):
        self.min_accuracy = min_accuracy
        
    def initialize(self):
        self.critical_epoch_ = -1
    
    # runst after each epoch
    def on_epoch_end(self, net, **kwargs):
        if self.critical_epoch_ > -1:
            return
        
        if net.history[-1, 'valid_acc'] >= self.min_accuracy:
            self.critical_epoch_ = len(net.history)
            
    # runs after each training
    def on_train_end(self, net, **kwargs):
        if self.critical_epoch_ < 0:
            msg = f"Accuracy never reached {self.min_accuracy}"
        else:
            msg = f"Accuracy reached {self.min_accuracy} at epoch {self.critical_epoch_}"
            
        tweet(msg)

Train the model

In [30]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=15,
    lr=0.02,
    warm_start=True,
    callbacks=[AccuracyTweet(min_accuracy=0.7)],
)

net.fit(X, y)
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7250       0.5050        0.7168  0.0168
      2        0.7169       0.5000        0.7069  0.0112
      3        0.7091       0.5150        0.6997  0.0125
      4        0.6983       0.5250        0.6938  0.0080
      5        0.6949       0.5650        0.6886  0.0056
      6        0.6905       0.5850        0.6849  0.0052
      7        0.6806       0.6050        0.6812  0.0056
      8        0.6845       0.6500        0.6776  0.0052
      9        0.6811       0.6550        0.6743  0.0060
     10        0.6742       0.6700        0.6712  0.0059
     11        0.6707       0.6900        0.6675  0.0053
     12        0.6706       0.6950        0.6641  0.0059
     13        0.6701       0.7100        0.6612  0.0056
     14        0.6615       0.7100        0.6575  0.0054
     15        0.6633       0.7350        0.6537  0.0060
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  ),
)

Multiple return values from forward

In [31]:
from skorch import NeuralNetRegressor

class Encoder(nn.Module):
    def __init__(self, num_units=5):
        super().__init__()
        self.num_units = num_units
        
        self.encode = nn.Sequential(
            nn.Linear(20, 10),
            nn.ReLU(),
            nn.Linear(10, num_units),
            nn.ReLU(),
        )
        
    def forward(self, X, **kwargs):
        return self.encode(X)
    
class Decoder(nn.Module):
    def __init__(self, num_units=5):
        super().__init__()
        self.num_units = num_units
        
        self.decode = nn.Sequential(
            nn.Linear(self.num_units, 10),
            nn.ReLU(),
            nn.Linear(10, 20),
        )
        
    def forward(self, X, **kwargs):
        return self.decode(X)
    
class AutoEncoder(nn.Module):
    def __init__(self, num_units=5):
        super().__init__()
        self.num_units = num_units
        
        self.encoder = Encoder(self.num_units)
        self.decoder = Decoder(self.num_units)
        
    def forward(self, X, **kwargs):
        encoded = self.encoder(X)
        decoded = self.decoder(encoded)
        return decoded, encoded

Override the get_loss method

In [32]:
class AutoEncoderNet(NeuralNetRegressor):
    def get_loss(self, y_pred, y_true, *args, **kwargs):
        decoded, encoded = y_pred
        
        loss_reconstruction = super().get_loss(
            decoded, y_true, *args, **kwargs)
        
        loss_l1 = torch.abs(encoded).sum()
        
        return loss_reconstruction + loss_l1

Training the autoencoder

In [41]:
net = AutoEncoderNet(
    AutoEncoder,
    module__num_units=5,
    lr=0.3,
    max_epochs=20
)

net.fit(X, X)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1       29.4691        3.8066  0.0164
      2        3.8723        3.7768  0.0141
      3        3.8420        3.7617  0.0105
      4        3.8259        3.7545  0.0092
      5        3.8178        3.7510  0.0095
      6        3.8139        3.7494  0.0073
      7        3.8120        3.7485  0.0099
      8        3.8110        3.7480  0.0099
      9        3.8105        3.7476  0.0093
     10        3.8103        3.7474  0.0080
     11        3.8101        3.7472  0.0110
     12        3.8100        3.7471  0.0083
     13        3.8100        3.7470  0.0072
     14        3.8099        3.7470  0.0126
     15        3.8099        3.7469  0.0077
     16        3.8099        3.7469  0.0074
     17        3.8099        3.7469  0.0067
     18        3.8099        3.7469  0.0063
     19        3.8099        3.7468  0.0068
     20        3.8099        3.7468  0.0064


<class '__main__.AutoEncoderNet'>[initialized](
  module_=AutoEncoder(
    (encoder): Encoder(
      (encode): Sequential(
        (0): Linear(in_features=20, out_features=10, bias=True)
        (1): ReLU()
        (2): Linear(in_features=10, out_features=5, bias=True)
        (3): ReLU()
      )
    )
    (decoder): Decoder(
      (decode): Sequential(
        (0): Linear(in_features=5, out_features=10, bias=True)
        (1): ReLU()
        (2): Linear(in_features=10, out_features=20, bias=True)
      )
    )
  ),
)

Extracting decoder and encoder output

In [42]:
y_pred = net.predict(X)
print(y_pred.shape) # only decoder state is returned

(1000, 20)


In [43]:
# retrieve all predicted batches from Module.forward
decoder_pred, encoder_pred = net.forward(X)
print(decoder_pred.shape, encoder_pred.shape)

torch.Size([1000, 20]) torch.Size([1000, 5])


In [36]:
# lazy colleciton, one batch at a time
for decoder_pred, encoder_pred in net.forward_iter(X):
    print(decoder_pred.shape, encoder_pred.shape)
    break

torch.Size([128, 20]) torch.Size([128, 5])


Was the encoder sparse?

In [44]:
torch.isclose(encoder_pred, torch.zeros_like(encoder_pred)).float().mean()

tensor(1.)